## Install packages

In [5]:
!poetry install --no-root

/bin/bash: line 1: poetry: command not found


## OPENAI settings

In [6]:
import os
os.environ['OPENAI_API_KEY'] = <openai-api-key>

## LLM

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)
name = llm.predict("I want to open a restaurant for Indian food. Suggest a fency name for this.")
print(name)

## Prompt template

In [11]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)

I want to open a restaurant for Italian food. Suggest a fency name for this.


## Chains

In [14]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Hungarian")

'\n\n"Magyar Manor: A Taste of Hungary" '

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Hungarian")

### Complete code part

In [ ]:
llm = OpenAI()